#Особенности Apache Spark Dataframes
Изучите теоретическую часть.

In [1]:
# Устанавливаем OpenJDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Закачиваем Spark
!wget -q http://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz -O spark.tgz
# Распаковываем архив со Spark
!tar xf spark.tgz
# Устанавливаем пакет findspark для работы со Spark из Python
!pip install -q findspark
# Настраиваем переменные окружения для работы с Apache Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"
# Находим установку Spark
import findspark
findspark.init()
# Подключаем необходимые модули для работы со Spark из Python
from pyspark.sql import SparkSession
# Создаем сессию Spark на локальном компьютере
spark = SparkSession.builder.master("local[*]").getOrCreate()
!wget https://datahub.io/core/population/r/population.json -O population.json -q

##Задача
Модернизируйте заготовку заменив все участки ```<put your code here>``` на ваш код для того, что бы:
* вычислить прирост населения в каждой стране от года к году (воспользуйтесь функцией [lag](https://spark.apache.org/docs/2.3.0/api/sql/#lag) или [lead](https://spark.apache.org/docs/2.3.0/api/sql/#lead))
* вычислить средний прирост за период с 1990 по 2018 год
* вычислить и вывести на экран страны с трендом на убыль населения с 1990 по 2018 годы


Пример вывода на экран:
```
+------------+-------------------+
|Country Name|              trend|
+------------+-------------------+
|    Bulgaria|-14276.898305084746|
|     Hungary|-3532.2542372881358|
|      Latvia| -3284.830508474576|
+------------+-------------------+
```

In [29]:
from pyspark.sql.functions import col, lag, lit, avg
from pyspark.sql import Window
df = spark.read.json("/content/population.json")
windowCountry = Window.partitionBy("Country Name").orderBy("Year")
df = df.withColumn('lag', lag(col('Value'), 1,).over(windowCountry))
df = df.withColumn('growth_pop', col('Value') - col('lag'))
df.show()
df = df.where('year > 1989')
df = df.where('year < 2019')
df = df.groupBy('Country Name').agg(avg('growth_pop').alias('trend'))
df.show()


+------------+------------+-----------+----+-----------+----------+
|Country Code|Country Name|      Value|Year|        lag|growth_pop|
+------------+------------+-----------+----+-----------+----------+
|         AFG| Afghanistan|  8996973.0|1960|       null|      null|
|         AFG| Afghanistan|  9169410.0|1961|  8996973.0|  172437.0|
|         AFG| Afghanistan|  9351441.0|1962|  9169410.0|  182031.0|
|         AFG| Afghanistan|  9543205.0|1963|  9351441.0|  191764.0|
|         AFG| Afghanistan|  9744781.0|1964|  9543205.0|  201576.0|
|         AFG| Afghanistan|  9956320.0|1965|  9744781.0|  211539.0|
|         AFG| Afghanistan|1.0174836E7|1966|  9956320.0|  218516.0|
|         AFG| Afghanistan|1.0399926E7|1967|1.0174836E7|  225090.0|
|         AFG| Afghanistan|1.0637063E7|1968|1.0399926E7|  237137.0|
|         AFG| Afghanistan|1.0893776E7|1969|1.0637063E7|  256713.0|
|         AFG| Afghanistan|1.1173642E7|1970|1.0893776E7|  279866.0|
|         AFG| Afghanistan|1.1475445E7|1971|1.11

In [30]:
df.where('trend < 0').show()

+--------------------+-------------------+
|        Country Name|              trend|
+--------------------+-------------------+
|             Albania|-12467.827586206897|
|             Armenia|           -20162.0|
|             Belarus| -23672.44827586207|
|Bosnia and Herzeg...| -40823.96551724138|
|            Bulgaria|  -63859.8275862069|
|Central Europe an...| -285852.3448275862|
|             Croatia|-23428.172413793105|
|             Estonia| -8488.068965517241|
|             Georgia| -37129.34482758621|
|             Hungary|-24350.172413793105|
|              Latvia|-25509.689655172413|
|           Lithuania|-30438.344827586207|
|             Moldova| -8655.103448275862|
|         Puerto Rico|-10706.793103448275|
|             Romania|-127424.58620689655|
|  Russian Federation|-111825.86206896552|
|              Serbia| -21549.85714285714|
|             Ukraine|-246568.41379310345|
+--------------------+-------------------+

